## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

## Best Practices
- Preprocessing and Cleaning (Feature engineering)
- Train Test Split
- BOW, TFIDF, Word2Vec
- Train ML Algorithms

In [1]:
# Load Dataset
import pandas as pd
df = pd.read_csv('all_kindle_review.csv')

In [2]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
df.shape

(12000, 2)

In [6]:
df.isnull().sum()

,0
reviewText,0
rating,0


In [7]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [8]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [10]:
# Preprocessing and cleaning
# Positive review is 1 and negative review is 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [11]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [13]:
# 1. lower all cases
df['reviewText'] = df['reviewText'].str.lower()

In [15]:
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# 2. Clean the data ---> removing special characters, stopwords, punctuations

# Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
# Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
# Remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
# Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
# Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


<ipython-input-16-985189cffbf1>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [17]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [18]:
# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [19]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [22]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [23]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [24]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.2, random_state=42)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [31]:
from sklearn.naive_bayes import GaussianNB
nb_bow = GaussianNB()
nb_bow.fit(X_train_bow, y_train)

GaussianNB()

In [32]:
nb_tfidf = GaussianNB()
nb_tfidf.fit(X_train_tfidf, y_train)

GaussianNB()

In [35]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_pred_bow = nb_bow.predict(X_test_bow)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)

In [36]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.575


In [37]:
confusion_matrix(y_test,y_pred_bow)

array([[500, 303],
       [717, 880]])

In [38]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5791666666666667


In [39]:
confusion_matrix(y_test,y_pred_tfidf)

array([[489, 314],
       [696, 901]])

In [40]:
# trying word2vec
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [42]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
vec_king = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [44]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
# another way to do this
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

words = []
for sent in df['reviewText']:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

In [46]:
# train word2vec from scratch
model = gensim.models.Word2Vec(words, epochs=5)

In [47]:
model.corpus_count

12000

In [48]:
model.wv.similar_by_word('kid')

[('hed', 0.9524778723716736),
 ('approve', 0.9480751752853394),
 ('asked', 0.9428568482398987),
 ('shed', 0.9425236582756042),
 ('allow', 0.9385542869567871),
 ('truth', 0.934225857257843),
 ('pretend', 0.9336597919464111),
 ('favor', 0.9331308007240295),
 ('neglecting', 0.9331230521202087),
 ('hasnt', 0.9325833320617676)]

In [49]:
import numpy as np

def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)

    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [50]:
from tqdm import tqdm

#apply for every sentences
X = []
for i in tqdm(range(len(words))):
  X.append(avg_word2vec(words[i]))

100%|██████████| 12000/12000 [00:38<00:00, 311.45it/s]


In [51]:
df1 = pd.DataFrame()
for i in range(0,len(X)):
  df1 =  pd.concat([df1,pd.DataFrame(X[i].reshape(1,-1))],ignore_index = True)

In [52]:
df1.shape

(12000, 100)

In [53]:
df1['rating'] = df['rating']

In [54]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,rating
0,-0.134232,0.301416,0.192619,0.137062,0.056049,-0.400808,0.087738,0.637819,-0.225884,-0.283104,...,0.188753,-0.149247,0.111611,0.569182,0.114615,-0.093372,-0.302223,0.042030,0.153761,1
1,-0.218912,0.366641,-0.048438,0.220308,0.344153,-0.546966,0.291599,0.687881,-0.359364,-0.413296,...,0.428710,-0.318362,0.084770,0.315006,0.231498,0.027281,-0.018374,-0.034871,0.150349,1
2,-0.208291,0.369294,0.128723,0.139945,0.234897,-0.610892,0.292970,0.707842,-0.323094,-0.343452,...,0.313966,-0.124167,-0.022321,0.359893,0.183828,0.078960,-0.096230,-0.089202,0.182614,1
3,-0.387400,0.375959,0.188572,0.233522,0.274913,-0.734945,0.234496,0.662641,-0.090076,-0.164625,...,0.221926,-0.116477,-0.328051,0.047822,0.191448,0.443692,0.116082,-0.099335,0.136580,1
4,0.062979,0.528272,0.042806,-0.084317,0.055262,-0.603635,0.339799,0.911574,-0.189788,-0.480679,...,0.296186,0.133478,-0.003772,0.334816,0.117125,-0.106080,-0.165462,-0.046632,-0.009084,1


In [61]:
X = df1.drop(['rating'], axis=1)
y = df1['rating']

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
9182,-0.145027,0.554270,0.292654,0.072023,0.069058,-0.680314,0.324954,0.739236,-0.214972,-0.579586,...,0.573697,0.213762,-0.039637,-0.127016,0.257521,0.200781,0.115425,-0.175011,-0.019957,0.093708
11091,-0.109534,0.515171,0.279537,-0.149610,0.089357,-0.537589,0.371198,0.735365,-0.410641,-0.781752,...,0.492129,0.291256,0.270095,0.122707,0.343741,0.217026,0.046983,-0.314409,-0.061494,0.174745
6428,-0.076485,0.415203,0.173149,0.060244,0.046463,-0.436125,0.194128,0.644940,-0.203029,-0.246916,...,0.437586,0.259115,-0.041667,0.055918,0.413159,0.004345,-0.052712,-0.191084,0.053093,0.138989
288,-0.219298,0.371993,-0.004978,0.305376,0.199350,-0.596779,0.352015,0.718288,-0.303207,-0.387632,...,0.609844,0.342750,-0.261229,0.085459,0.406025,0.145040,-0.109844,0.004440,0.017862,0.103484
2626,-0.219914,0.287799,0.216478,0.110907,0.103171,-0.477135,0.183549,0.616640,-0.169499,-0.220526,...,0.379832,0.156936,-0.085895,0.020018,0.440215,0.155964,0.084357,-0.264123,0.026021,0.140729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,0.095282,0.808973,0.121887,-0.255458,-0.005519,-0.504718,0.600901,0.952527,-0.525332,-1.020268,...,0.715878,0.480378,0.434767,0.136141,0.395074,0.137133,-0.045129,-0.257441,-0.002853,0.006593
5191,0.171540,0.280360,0.176907,-0.042343,0.168744,-0.352697,0.202303,0.688695,-0.437702,-0.649412,...,0.450588,0.391350,0.057197,0.157229,0.380943,0.206872,-0.067338,-0.163478,-0.090242,0.048935
5390,-0.153778,0.413348,0.209430,0.100439,0.117051,-0.490415,0.259510,0.706300,-0.358510,-0.481480,...,0.481444,0.264245,-0.042015,0.098143,0.409487,0.109400,0.004850,-0.231486,0.018924,0.074113
860,-0.206594,0.250756,0.035232,0.320173,0.128020,-0.463907,0.177982,0.641272,-0.129087,-0.081277,...,0.538737,0.273027,-0.330261,0.021863,0.562737,0.152534,0.036732,-0.127397,0.068951,0.175982


In [64]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1935,0.017959,0.527229,0.177294,0.106195,0.333469,-0.430265,0.458967,0.669604,-0.163873,-0.697908,...,0.394019,0.447632,0.027782,-0.016346,0.057542,0.230630,0.167942,0.185679,-0.392355,0.162911
6494,0.139875,0.609080,0.132266,-0.094863,-0.119000,-0.403820,0.161357,0.619989,-0.430291,-0.607087,...,0.606911,0.166995,0.164744,0.113162,0.505742,0.020305,-0.075150,-0.511378,0.076685,-0.113961
1720,-0.261662,0.585001,0.180138,0.259419,0.301035,-0.673048,0.488595,0.771070,-0.115768,-0.695222,...,0.426151,0.394414,-0.144459,-0.092207,0.146829,0.200857,0.190157,0.054829,-0.238528,0.060956
9120,-0.391502,0.455800,0.113709,0.297960,0.237108,-0.620639,0.414356,0.706334,-0.221196,-0.367356,...,0.723021,0.324489,-0.264448,-0.026663,0.358784,0.170368,0.013924,0.016377,-0.029982,0.271495
360,-0.094494,0.428470,0.060755,0.093305,0.228761,-0.482572,0.247510,0.681598,-0.258992,-0.379371,...,0.519303,0.310499,-0.194265,-0.004593,0.188457,0.116446,-0.117233,0.019977,0.030074,0.063747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.175626,0.580278,0.029725,-0.047308,0.264605,-0.385824,0.066691,0.944870,-0.040403,-0.587572,...,0.493636,0.530600,0.031884,0.070593,0.335050,0.056464,0.020915,-0.015426,-0.022317,-0.198776
11877,-0.240485,0.422952,-0.010373,-0.003644,0.377272,-0.624055,0.427148,0.758844,-0.402267,-0.482540,...,0.675524,0.379134,-0.032540,-0.005295,0.341847,0.235650,0.110187,-0.139523,-0.136084,0.217823
5421,-0.116106,0.234491,0.226714,0.056968,0.275504,-0.410345,0.062717,0.735983,-0.308649,-0.376975,...,0.461319,0.307304,-0.048143,0.122829,0.640937,0.150202,-0.000451,-0.285688,-0.008627,0.251770
3855,-0.279259,0.644095,0.042327,-0.136882,0.361875,-0.658034,0.571850,0.744792,-0.405068,-0.685761,...,0.676978,0.485252,0.155622,-0.129667,0.237123,0.303115,0.358849,-0.136506,-0.148590,0.095199


In [65]:
y_train

,rating
9182,1
11091,0
6428,1
288,0
2626,1
...,...
11964,0
5191,1
5390,0
860,1


In [66]:
y_test

,rating
1935,1
6494,0
1720,0
9120,0
360,0
...,...
1195,1
11877,1
5421,1
3855,1


In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve

In [68]:
models={
    "Logisitic Regression":LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Gradient Boost":GradientBoostingClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Gausian Naive Bayes': GaussianNB()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)


    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc


    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))

    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))



    print('----------------------------------')

    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))


    print('='*35)
    print('\n')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logisitic Regression
Model performance for Training set
- Accuracy: 0.7809
- F1 score: 0.7733
- Precision: 0.8032
- Recall: 0.8896
- Roc Auc Score: 0.7265
----------------------------------
Model performance for Test set
- Accuracy: 0.7812
- F1 score: 0.7727
- Precision: 0.7998
- Recall: 0.8954
- Roc Auc Score: 0.7248


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.6958
- F1 score: 0.6980
- Precision: 0.7802
- Recall: 0.7558
- Roc Auc Score: 0.6662


Random Forest
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.7675
- F1 score: 0.7611
- Precision: 0.7984
- Recall: 0.8704
- Roc Auc Score: 0.7166


Gradient Boost
Model performance for Training se